## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets
import numpy as np

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)



In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("WeatherPy_vacation.csv")
vacation_df.head()


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Altamira,BR,81.34,broken clouds,-3.2033,-52.2064,Requinte Hotel
1,8,Hithadhoo,MV,83.03,few clouds,-0.6000,73.0833,Scoop Guest House
2,14,Aguada De Pasajeros,CU,75.74,broken clouds,22.3850,-80.8508,CASA MAMÁ
3,19,Coahuayana,MX,76.62,overcast clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
4,20,Andrews,US,78.06,clear sky,32.3187,-102.5457,Days Inn by Wyndham Andrews Texas


In [3]:
params = {
    "radius": 2000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in vacation_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        vacation_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found.... skipping")
        

Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not foun

In [7]:
clean_hotel_df = vacation_df.head(10)

In [8]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature 
#for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
print(locations)

       Lat       Lng
0  -3.2033  -52.2064
1  -0.6000   73.0833
2  22.3850  -80.8508
3  18.7333 -103.6833
4  32.3187 -102.5457
5  -0.7193    8.7815
6   3.0833  -59.9500
7  -8.8978  -35.1478
8  59.7035   30.7861
9  35.1333  140.3000


In [9]:
# 4a. Add a marker layer for each city to the map.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [32]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df['Country']=='MX'].head(1)
vacation_end = vacation_start
vacation_stop1 = vacation_df.loc[(vacation_df['Country']=='MX') & (vacation_df['City'] != vacation_start['City'].values[0])].head(1)
vacation_stop2 = vacation_df.loc[(vacation_df['Country']=='MX') & (vacation_df['City'] != vacation_start['City'].values[0])\
                                & (vacation_df['City'] != vacation_stop1['City'].values[0])].head(1) 
vacation_stop3 = vacation_df.loc[(vacation_df['Country']=='MX') & (vacation_df['City'] != vacation_start['City'].values[0])\
                                & (vacation_df['City'] != vacation_stop1['City'].values[0]) \
                                                 & (vacation_df['City'] != vacation_stop2['City'].values[0])].head(1)


In [37]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = pd.DataFrame(vacation_start, columns=['Lat', 'Lng']).to_records(index=False)
end = pd.DataFrame(vacation_end, columns=['Lat', 'Lng']).to_records(index=False)
stop1 = pd.DataFrame(vacation_stop1, columns=['Lat', 'Lng']).to_records(index=False)
stop2 = pd.DataFrame(vacation_stop2, columns=['Lat', 'Lng']).to_records(index=False)
stop3 = pd.DataFrame(vacation_stop3, columns=['Lat', 'Lng']).to_records(index=False)

print(start)
print(end)
print(stop1)
print(stop2)
print(stop3)

[(18.7333, -103.6833)]
[(18.7333, -103.6833)]
[(21.5167, -105.2667)]
[(15.7432, -96.4661)]
[(22.8909, -109.9124)]


In [66]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
routemap = gmaps.directions_layer(start[0],end[0], waypoints=[stop1[0],stop2[0],stop3[0]],travel_mode='DRIVING')
fig.add_layer(routemap)
fig


Figure(layout=FigureLayout(height='420px'))

In [67]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end],ignore_index=True)
itinerary_df


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,19,Coahuayana,MX,76.62,overcast clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
1,115,San Blas,MX,78.66,overcast clouds,21.5167,-105.2667,Ruinas Hotel playa Hermosa
2,242,Pochutla,MX,76.82,overcast clouds,15.7432,-96.4661,Hotel Posada San Jose
3,275,Cabo San Lucas,MX,83.52,overcast clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
4,19,Coahuayana,MX,76.62,overcast clouds,18.7333,-103.6833,El rranchito michuacan de ocampos


In [68]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [69]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(20.0,-100.0), zoom_level=4.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))